# Working with data in delta format from OneLake in Azure Machine Learning

If you are planning to read data from OneLake in Azure Machine Learning, most of the time you would need to work with the MLtable data asset type. To work with MLtable, use the `mltable` and the `azureml-dataprep` packages:

In [ ]:
%pip install -U mltable azureml-dataprep[pandas] --quiet

## Read data from OneLake in delta format using MLtable

To read data from OnaLake, go to the asset or table you want to read and copy the ABFS path to it. Remember that delta also support Time Travel, which allows you to read the data at specific points in time. Use a timestamp to indicate that:

In [2]:
import time

delta_table_path = "abfss://<GUID>@onelake.dfs.fabric.microsoft.com/<GUID>/Tables/phone_device"
timestamp_as_of = time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

Let's use this information to read the data into an MLtable:

In [3]:
import mltable

tbl = mltable.from_delta_lake(delta_table_uri=delta_table_path, timestamp_as_of=timestamp_as_of)

We can see if we can successfully read the data:

In [4]:
tbl.to_pandas_dataframe()

Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


,brand,model,network_technology,2G_bands,3G_bands,4G_bands,network_speed,GPRS,EDGE,announced,...,bluetooth,GPS,NFC,radio,USB,sensors,battery,colors,approx_price_EUR,img_url
0,Asus,Google Nexus 7 (2013),GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - all versions,HSDPA 850 / 900 / 1700 / 1900 / 2100,LTE band 1(2100)| 2(1900)| 3(1800)| 4(1700/210...,HSPA LTE,Yes,Yes,2013 July,...,Wi-Fi 802.11 a/b/g/n| dual-band,4.0| A2DP| LE,Yes with A-GPS,Yes,No,microUSB 2.0 (SlimPort),Accelerometer| gyro| proximity| compass,Non-removable Li-Ion 3950 mAh battery,Black,230
1,HTC,10 Lifestyle,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 5(850)| 7(2600)| 8(...,HSPA LTE Cat6 300/50 Mbps,Yes,Yes,2016 April,...,4.2| A2DP| aptX,Yes with A-GPS GLONASS/ BDS (region dependent),Yes,No,2.0| Type-C 1.0 reversible connector,Fingerprint (front-mounted)| accelerometer| gy...,Non-removable Li-Ion 3000 mAh battery,Carbon Gray| Topaz Gold| Glacier Silver,420,http://cdn2.gsmarena.com/vv/bigpic/htc-10-life...
2,HTC,10,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 5(850)| 7(2600)| 8(...,HSPA LTE-A (3CA) Cat9 450/50 Mbps,Yes,Yes,2016 April,...,4.2| A2DP| aptX,Yes with A-GPS GLONASS/ BDS (region dependent),Yes,No,3.1| Type-C 1.0 reversible connector,Fingerprint (front-mounted)| accelerometer| gy...,Non-removable Li-Ion 3000 mAh battery,Carbon Gray| Topaz Gold| Glacier Silver| Camel...,500,http://cdn2.gsmarena.com/vv/bigpic/htc-10-r.jpg
3,OnePlus,One,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1700 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 4(1700/2100)| 7(260...,HSPA 42.2/5.76 Mbps LTE Cat4 150/50 Mbps,Yes,Yes,2014 April,...,4.1| A2DP,Yes with A-GPS GLONASS,Yes,No,microUSB 2.0| USB Host,Accelerometer| gyro| proximity| compass,Non-removable Li-Po 3100 mAh battery,Silk White| Sandstone Black,250,http://cdn2.gsmarena.com/vv/bigpic/oneplus-one...
4,Samsung,Galaxy Tab A 9.7,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 5(850)| 7(2600)| 8(...,HSPA LTE Cat4 150/50 Mbps,Yes,Yes,2015 March,...,Wi-Fi 802.11 a/b/g/n| dual-band| WiFi Direct| ...,4.1| A2DP,Yes with A-GPS GLONASS,None,No,microUSB 2.0,Accelerometer,Non-removable Li-Ion 6000 mAh battery,Smoky Titanium| Smoky Blue| White,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8626,Sagem,MC 950,GSM,GSM 900 / 1800,None,None,None,No,No,2000,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc950b.gif
8627,Sagem,MC 959,GSM,GSM 900 / 1800,None,None,None,No,No,2000,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc959b.gif
8628,Sagem,MC 940,GSM,GSM 900 / 1800,None,None,None,No,No,2000,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc940b.gif
8629,Sagem,MC 932,GSM,GSM 900 / 1800,None,None,None,No,No,1999,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc932b.gif


## Register the data asset

MLtable can be registered as a data asset in Azure Machine Learning. Then, you can plug those assets to any pipeline you have. To create an MLtable adata asset, we need to save the file definition

Notice we are not saving a copy of the data, we are only saving an MLtable file with the definition of where the data is and how to read it.

In [5]:
tbl.save("phone-device")

Once saved, we can now use it to register the asset. 

In [6]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

VERSION = time.strftime("%Y-%m-%d-%H%M%S", time.gmtime())
phone_device_asset = Data(
    path="phone-device",
    type=AssetTypes.MLTABLE,
    description="A dataset containing multiple mobile phone information about their hardware and capabilities.",
    name="phone-device",
    version=VERSION,
)

> Pay attention about how `version` is indicated. Azure Machine Learning supports assigning any type of label to a version. In this case, we are associated to it a label with the point in time at which the delta table is pointing to. This ensure that when we see the version `2023-08-02` of the data asset, we know it provides data as what it was at that time.
> This feature allows reproducibility of results, which is paramount in an ML workload.

Let's first create an Azure Machine Learning workspace client:

In [7]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(DefaultAzureCredential())

Found the config file in: /config.json


Now, we can register the asset:

In [8]:
_ = ml_client.data.create_or_update(phone_device_asset)

Uploading phone-device (0.0 MBs): 100%|██████████| 597/597 [00:00<00:00, 8381.14it/s]




## Consuming data assets from notebooks

Once the data is registered as an asset, you can use it as any data asset in Azure Machine Learning, including in notebooks:

In [9]:
data_asset = ml_client.data.get(name="phone-device", label="latest")

> We are querying here the lat known version:

Notice how `mltable` library can access resources directly from the ID:

In [10]:
tbl_asset = mltable.load(f"azureml:/{data_asset.id}")

Let's check we can read the data:

In [11]:
tbl_asset.to_pandas_dataframe()

,brand,model,network_technology,2G_bands,3G_bands,4G_bands,network_speed,GPRS,EDGE,announced,...,bluetooth,GPS,NFC,radio,USB,sensors,battery,colors,approx_price_EUR,img_url
0,Asus,Google Nexus 7 (2013),GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - all versions,HSDPA 850 / 900 / 1700 / 1900 / 2100,LTE band 1(2100)| 2(1900)| 3(1800)| 4(1700/210...,HSPA LTE,Yes,Yes,2013 July,...,Wi-Fi 802.11 a/b/g/n| dual-band,4.0| A2DP| LE,Yes with A-GPS,Yes,No,microUSB 2.0 (SlimPort),Accelerometer| gyro| proximity| compass,Non-removable Li-Ion 3950 mAh battery,Black,230
1,HTC,10 Lifestyle,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 5(850)| 7(2600)| 8(...,HSPA LTE Cat6 300/50 Mbps,Yes,Yes,2016 April,...,4.2| A2DP| aptX,Yes with A-GPS GLONASS/ BDS (region dependent),Yes,No,2.0| Type-C 1.0 reversible connector,Fingerprint (front-mounted)| accelerometer| gy...,Non-removable Li-Ion 3000 mAh battery,Carbon Gray| Topaz Gold| Glacier Silver,420,http://cdn2.gsmarena.com/vv/bigpic/htc-10-life...
2,HTC,10,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 5(850)| 7(2600)| 8(...,HSPA LTE-A (3CA) Cat9 450/50 Mbps,Yes,Yes,2016 April,...,4.2| A2DP| aptX,Yes with A-GPS GLONASS/ BDS (region dependent),Yes,No,3.1| Type-C 1.0 reversible connector,Fingerprint (front-mounted)| accelerometer| gy...,Non-removable Li-Ion 3000 mAh battery,Carbon Gray| Topaz Gold| Glacier Silver| Camel...,500,http://cdn2.gsmarena.com/vv/bigpic/htc-10-r.jpg
3,OnePlus,One,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1700 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 4(1700/2100)| 7(260...,HSPA 42.2/5.76 Mbps LTE Cat4 150/50 Mbps,Yes,Yes,2014 April,...,4.1| A2DP,Yes with A-GPS GLONASS,Yes,No,microUSB 2.0| USB Host,Accelerometer| gyro| proximity| compass,Non-removable Li-Po 3100 mAh battery,Silk White| Sandstone Black,250,http://cdn2.gsmarena.com/vv/bigpic/oneplus-one...
4,Samsung,Galaxy Tab A 9.7,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,HSDPA 850 / 900 / 1900 / 2100,LTE band 1(2100)| 3(1800)| 5(850)| 7(2600)| 8(...,HSPA LTE Cat4 150/50 Mbps,Yes,Yes,2015 March,...,Wi-Fi 802.11 a/b/g/n| dual-band| WiFi Direct| ...,4.1| A2DP,Yes with A-GPS GLONASS,None,No,microUSB 2.0,Accelerometer,Non-removable Li-Ion 6000 mAh battery,Smoky Titanium| Smoky Blue| White,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8626,Sagem,MC 950,GSM,GSM 900 / 1800,None,None,None,No,No,2000,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc950b.gif
8627,Sagem,MC 959,GSM,GSM 900 / 1800,None,None,None,No,No,2000,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc959b.gif
8628,Sagem,MC 940,GSM,GSM 900 / 1800,None,None,None,No,No,2000,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc940b.gif
8629,Sagem,MC 932,GSM,GSM 900 / 1800,None,None,None,No,No,1999,...,No,No,None,No,None,None,Removable Li-Ion battery,None,None,http://cdn2.gsmarena.com/vv/bigpic/samc932b.gif
